Импортируем модуль sqlite3 для работы с базами данных.
Создаем два соединения с базами данных "netflix.sqlite" и "task1.sqlite".
Устанавливаем фабрика строк для базы данных "netflix.sqlite", которая возвращает словарь с полями и значениями для каждой строки результата запроса.

In [1]:
import sqlite3
netflix_db = sqlite3.connect('netflix.sqlite')
task_db = sqlite3.connect("task1.sqlite")
netflix_db.row_factory = lambda cursor, row: {col[0]: row[i] for i, col in enumerate(cursor.description)}

С помощью метода execute() выполняем запрос на создание таблицы films с полями film_id, type, title, director, country, date_added, release_year, rating, duration, listed_in и description.
С помощью метода execute() выполняем запрос на создание таблицы actors с полями actor_id и actor_name.
С помощью метода execute() выполняется запрос на создание таблицы actors_films с полями film_id и actor_id, а также устанавливаются ограничения PRIMARY KEY (actor_id, film_id) и FOREIGN KEY (film_id) REFERENCES films(film_id) и FOREIGN KEY (actor_id) REFERENCES actors(actor_id), чтобы связать таблицы между собой.

In [2]:
task_cur = task_db.cursor()
with task_db:
  task_cur.execute("""
  CREATE TABLE films (
    film_id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT,
    title TEXT,
    director TEXT,
    country TEXT,
    date_added TEXT,
    release_year INTEGER,
    rating TEXT,
    duration TEXT,
    listed_in TEXT,
    description TEXT
  );
  """)
  task_cur.execute("""
  CREATE TABLE actors (
    actor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    actor_name TEXT UNIQUE
  );
  """)
  task_cur.execute("""
    CREATE TABLE actors_films (
    film_id INTEGER,
    actor_id INTEGER,
    PRIMARY KEY (actor_id, film_id),
    FOREIGN KEY (film_id) REFERENCES films(film_id),
    FOREIGN KEY (actor_id) REFERENCES actors(actor_id)
  );
  """)

Задание 1

In [3]:
# Создаем курсоры для обоих баз данных.
netflix_cur = netflix_db.cursor()
task_cur = task_db.cursor()

# Создаеv переменнe. insert_query, в которой хранится запрос на добавление данных в таблицу films.
insert_query = """
INSERT INTO films (type, title, director, country, date_added, release_year, rating, duration, listed_in, description)
VALUES (:type, :title, :director, :country, :date_added, :release_year, :rating, :duration, :listed_in, :description);
"""

# Создаем переменную actors_films_insert_query, в которой хранится запрос на добавление данных в таблицу actors_films.
actors_films_insert_query = """
INSERT OR IGNORE INTO actors_films (film_id, actor_id)
VALUES (?, (SELECT actor_id FROM actors WHERE actor_name = ?));
"""

with netflix_db, task_db:
    # Итерация по строкам таблицы netflix_titles в базе данных netflix_db
    for row in netflix_cur.execute("SELECT * FROM netflix_titles"):
        # Выполнение запроса на вставку данных из текущей строки в таблицу films базы данных task_db
        task_cur.execute(insert_query, row)
        
        # Получение ID последней вставленной записи в таблицу films
        film_id = task_cur.execute("SELECT last_insert_rowid() FROM films;").fetchone()[0]
        
        # Итерация по актерам в колонке cast текущей строки
        for actor in row["cast"].split(", "):
            # Вставка актера в таблицу actors, игнорируя дубликаты
            task_cur.execute("INSERT OR IGNORE INTO actors (actor_name) VALUES (?)", (actor,))
            
            # Привязка актера к фильму в таблице actors_films
            task_cur.execute(actors_films_insert_query, (film_id, actor))

Задание 2:

In [4]:
with task_db:
    # Выполнение запроса для получения наиболее часто встречающейся пары актеров
    most_freq_pair = task_cur.execute("""
    SELECT a1.actor_name as name1, a2.actor_name as name2
    FROM actors_films af1
    JOIN actors_films af2 ON af1.film_id = af2.film_id AND af1.actor_id < af2.actor_id
    JOIN actors a1 ON af1.actor_id = a1.actor_id
    JOIN actors a2 ON af2.actor_id = a2.actor_id
    GROUP BY af1.actor_id, af2.actor_id
    ORDER BY COUNT(*) DESC
    LIMIT 1;
    """).fetchone()
most_freq_pair

('John Paul Tremblay', 'Robb Wells')